# Initialization

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from IPython.display import display, clear_output
from tqdm.auto import tqdm
import torch.nn.functional as F
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.profiler import profile, record_function, ProfilerActivity
import pickle
import os
import sys

ROOT = os.path.join("./")
sys.path.append(ROOT + "lib")
sys.path.append(ROOT + "models")

from helpers import *
from sourceset import SourceSet
from fourier import FourierModelTest, FourierModelTestNU
from transformer import FluxAnomalyPredictionTF
from general_trainer import compete, gen_train


device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)
torch.set_default_dtype(torch.float32)

/Users/matteo/.pyenv/versions/3.11.2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# with open(ROOT + "processed_datasets/transittrain.pt", "rb") as f:
#   data_train = torch.load(f, map_location=device)
# with open(ROOT + "processed_datasets/transitvalid.pt", "rb") as f:
#   data_valid = torch.load(f, map_location=device)

with open(ROOT + "processed_datasets/agglomerated_datatrain.pt", "rb") as f:
  data_train = torch.load(f, map_location=device)
with open(ROOT + "processed_datasets/agglomerated_datavalid.pt", "rb") as f:
  data_valid = torch.load(f, map_location=device)



data_train, data_valid

(<sourceset.SourceSet at 0x160e21790>, <sourceset.SourceSet at 0x160f3b390>)

# Model

In [3]:
model = FourierModelTest(100,3)
model_pars = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_pars])

print(params)

5701


In [4]:
train = DataLoader(data_train, batch_size=len(data_train), shuffle=True, collate_fn=padded_collate)
valid = DataLoader(data_valid, batch_size=len(data_valid), shuffle=False, collate_fn=padded_collate)

# Training Loop (Single)

In [5]:
kwargs=dict(samples=240, out=3)
model = FourierModelTest(**kwargs)
optim = torch.optim.Adam(model.parameters(), lr=0.003, weight_decay=0.00005)
loss = torch.nn.CrossEntropyLoss().to(device)
gen_train(model, kwargs, optim, loss, train, valid, 300, False)

: 

In [ ]:
validaccessor = list(valid)[0]
idx = 8
ex = (validaccessor[0][idx], validaccessor[1][idx])
print(ex[1])
plot_from_tensor(ex[0])
# print(ex[0])

tensor([1., 0., 0.])


or

In [ ]:
samples = 256
lr = 0.007
dft = FourierModelTest(samples,2)
nudft = FourierModelTestNU(samples,2)

optim1 = torch.optim.Adam(dft.parameters(), lr=lr, weight_decay=0.0009)
optim2 = torch.optim.Adam(nudft.parameters(), lr=lr*0.7, weight_decay=0.0012)

compete([("dft", dft, optim1), ("nudft", nudft, optim2)], 300, train, valid)

Training Progress:   0%|          | 0/300 [00:00<?, ?it/s]

RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [ ]:
model = FluxAnomalyPredictionTF(32, 0.1).to(device)


with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("dd"):
        ex = next(iter(train))
        out = model(ex[0])
        loss = loss_fn(out, ex[1])
        loss.backward()
        # for name, param in model.cpu().named_parameters():
        #   print(name, param.grad)
        # plot_grad_flow(model.cpu().named_parameters())

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

/Users/matteo/.pyenv/versions/3.11.2/lib/python3.11/site-packages/torch/autograd/profiler.py:179: UserWarning:

CUDA is not available, disabling CUDA profiling

STAGE:2024-01-04 21:16:18 67123:5609078 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2024-01-04 21:16:18 67123:5609078 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2024-01-04 21:16:18 67123:5609078 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


NameError: name 'loss_fn' is not defined